In [1]:
import tensorflow as tf
import numpy as np
import functions as func
from matplotlib import pyplot as plt

In [2]:
# load data
data_x = np.load('../cifar_x.npy')
data_y = np.load('../cifar_y.npy')
test_x = np.load('../cifar_test_x.npy')
test_y = np.load('../cifar_test_y.npy')
index = 0

In [3]:
# Parameters
learning_rate = 0.001
training_iters = 300000
batch_size = 128
mini_batch = 8
train_iter = batch_size//mini_batch
display_step = 10

# Network Parameters
H = 32
W = 32
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, H, W, 3])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [4]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 3, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.ones([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([8*8*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.ones([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
para = [weights['wc1'], weights['wc2'], weights['wd1'], weights['out'],
        biases['bc1'], biases['bc2'], biases['bd1'], biases['out']]

In [5]:
# body of network
def body2(i, x, out):
    # Convolution Layer
    inputx = x.read(index=i)
    conv1 = func.conv2d(inputx, weights['wc1'], biases['bc1'])
    
    # Pooling (down-sampling)
    p1 = func.extract_patches(conv1, 'SAME', 2, 2)
    f1 = func.majority_frequency(p1)
    #     maxpooling
    pool1 = func.max_pool(p=p1)
    # Convolution Layer
    conv2 = func.conv2d(pool1, weights['wc2'], biases['bc2'])
    
    out = out.write(index=i, value=conv2)
    i += 1
    return i, x, out

In [6]:
def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

In [7]:
# Create model
def conv_net(x, weights, biases, dropout):
    # Convolution Layer
    conv1 = func.conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    pool1 = maxpool2d(conv1, k=2)
    # Convolution Layer
    conv2 = func.conv2d(pool1, weights['wc2'], biases['bc2'])
    out = conv2
    return out

In [8]:
inputxs = tf.TensorArray(dtype=tf.float32, size=train_iter, clear_after_read=True).split(x, np.array([mini_batch] * train_iter))
out = tf.TensorArray(dtype=tf.float32, size=train_iter)

In [9]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
# opt = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [10]:
pred

<tf.Tensor 'Relu_1:0' shape=(?, 16, 16, 64) dtype=float32>

In [11]:
# compute gradient and update the weights
i0 = tf.constant(0)
con = lambda i, x, g: i < train_iter

i0, inputx, out = tf.while_loop(cond=con, body=body2, loop_vars=[i0, inputxs, out], parallel_iterations=1)

pred1 = tf.reshape(out.stack(), [128, 16,16,64])

cost1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))


In [12]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
with tf.Session(config=config) as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
#     while step < 2:
        batch_x, batch_y, index = func.get_batch(batch_size, data_x, data_y, index)
        # Run optimization op (backprop)
        
        
        p = sess.run(pred, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        p1 = sess.run(pred1, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        
        step += 1
        print(np.max(np.abs(p - p1)))
    
    print("Optimization Finished!")

0.0107422
0.0078125
0.00976562
0.0078125
0.00830078
0.00878906
0.00830078
0.00976562
0.0078125


KeyboardInterrupt: 

In [14]:
p1

array([[[[  2.23317838,   8.31939125,   0.        , ...,   5.4193058 ,
            5.26049566,   3.96007395],
         [  0.        ,   0.        ,   0.        , ...,  31.36197281,
            2.09435606,   0.        ],
         [  0.        ,   6.10947752,   0.        , ...,  22.66752243,
           13.73317719,   0.        ],
         ..., 
         [  4.94648075,   4.97213936,   5.28604746, ...,   0.        ,
            0.        ,   8.96533394],
         [  2.90176225,   2.66871548,   4.54663134, ...,   1.40262294,
            1.67453182,   0.        ],
         [  4.85346508,   2.43643451,   0.43951425, ...,   3.03064823,
            3.41016769,   0.        ]],

        [[  2.35889983,   0.43626356,   0.        , ...,   3.4771452 ,
            1.24453199,  14.18448544],
         [  0.        ,   5.78289509,   0.        , ...,  23.94904327,
            3.66562462,   1.75739586],
         [  4.86962271,   0.        ,   0.        , ...,  20.68817139,
           12.16908646,   0.    

In [ ]:
pred